In [1]:
# --- 0. Install necessary library ---
#changed as recommended approach (!pip)-->(%pip) which equals to: python -m pip install <module>
%pip install gdown pandas numpy scikit-learn matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler # Keep for potential future use
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import gdown # Library to download from Google Drive
from sklearn.impute import SimpleImputer # Import imputer

In [ ]:

# --- 1. Download Data from Google Drive ---
file_id = '1dRE9RSdR3pCnDbr5iYHfH2sdR4hOdG3H'
output_file = 'synthetic_data_loaded_with_header.csv' # Changed filename for clarity
url = f'https://drive.google.com/uc?id={file_id}'

print(f"Downloading file from Google Drive (ID: {file_id})...")
try:
    # Using fuzzy=True might help if the direct download link changes slightly
    gdown.download(url, output_file, quiet=False, fuzzy=True)
    print(f"File saved as {output_file}")
except Exception as e:
    print(f"Failed to download file: {e}")
    exit()

In [3]:
#Alternativelly use the combined SECOM dataset csv in the dataset subfolder
# Define the dataset folder path (the files are in a subfolder called dataset)
from pathlib import Path

dataset_folder = Path('dataset')
dataset_filename= 'SECOM_combined_dataset.csv'

print(f"Attempting to open the combined file {dataset_filename} saved locally in the folder {dataset_folder}")
try:
     output_file = dataset_folder / dataset_filename
     print(f'File successfully opened.')
except FileNotFoundError as e:
     print(f'File not found: {e}')
except Exception as e:
     print(e)

Attempting to open the combined file SECOM_combined_dataset.csv saved locally in the folder dataset
File successfully opened.


In [4]:
# --- 2. Load Data using Pandas (Assuming Header IS Present) ---
try:
    # FIX: Remove header=None to let Pandas read the first row as header
    data = pd.read_csv(output_file)
    print(f"Data loaded successfully from {output_file} with shape: {data.shape}")
    # Display first few rows and info to check data types
    print("\nFirst 5 rows of loaded data:")
    print(data.head())
    print("\nData info:")
    data.info()
except Exception as e:
    print(f"Failed to load data from {output_file}: {e}")
    exit()

Data loaded successfully from dataset\SECOM_combined_dataset.csv with shape: (1567, 591)

First 5 rows of loaded data:
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    3030.93    2564.00  2187.7333  1411.1265     1.3602      100.0   
1    3095.78    2465.14  2230.4222  1463.6606     0.8294      100.0   
2    2932.61    2559.94  2186.4111  1698.0172     1.5102      100.0   
3    2988.72    2479.90  2199.0333   909.7926     1.3204      100.0   
4    3032.24    2502.87  2233.3667  1326.5200     1.5334      100.0   

   feature_6  feature_7  feature_8  feature_9  ...  feature_581  feature_582  \
0    97.6133     0.1242     1.5005     0.0162  ...          NaN       0.5005   
1   102.3433     0.1247     1.4966    -0.0005  ...     208.2045       0.5019   
2    95.4878     0.1241     1.4436     0.0041  ...      82.8602       0.4958   
3   104.2367     0.1217     1.4882    -0.0124  ...      73.8432       0.4990   
4   100.3967     0.1235     1.5031    -0.0031  ...    

In [5]:
# --- 3. Prepare Data ---
if data.shape[1] > 1:
    # Assume last column is target 'y', all others are features 'X'
    feature_names = data.columns[:-1].tolist() # Get actual feature names from header
    X_df = data[feature_names] # Select feature columns using names
    y_series = data.iloc[:, -1] # Select target column by position

    print(f"\nFeature names extracted from header: {feature_names[:5]}... (Total: {len(feature_names)})")

    # --- Keep Robust Cleaning Steps ---
    # Convert non-numeric strings to numeric, coercing errors, otherwise-> NaN
    X_df_numeric = X_df.apply(pd.to_numeric, errors='coerce')
    y_series_numeric = pd.to_numeric(y_series, errors='coerce')

    # Handle potential NaN values resulting from coercion or missing values in original file
    imputer_X = SimpleImputer(strategy='mean')
    X_imputed = imputer_X.fit_transform(X_df_numeric)

    missing_labels_count = y_series_numeric.isna().sum()
    
    imputer_y = SimpleImputer(strategy='most_frequent') #changed strategy from mean to most frequent since we have binary labels
    y_imputed = imputer_y.fit_transform(y_series_numeric.values.reshape(-1, 1)).flatten()

    # Check for NaNs after imputation (should only happen if a whole column was non-numeric/NaN)
    if np.isnan(X_imputed).any() or np.isnan(y_imputed).any():
        print("Warning: NaNs still present after imputation. Check columns with all invalid values.")
        # Consider more advanced imputation or dropping problematic columns/rows if this occurs

    X = X_imputed
    y = y_imputed

    print(f"Features shape after cleaning: {X.shape}")
    
    print(f"Number of missing labels to be imputed: {missing_labels_count}")
    print(f"Target shape after cleaning: {y.shape}")
    n_features_loaded = X.shape[1] # Keep track of the number of features

else:
    print("Error: Loaded data has only one column. Cannot separate features and target.")
    exit()


Feature names extracted from header: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']... (Total: 590)
Features shape after cleaning: (1567, 590)
Number of missing labels to be imputed: 0
Target shape after cleaning: (1567,)


In [6]:

# --- 4. Split Data for Fitness Evaluation ---
# Using the cleaned X and y, with stratify helps maintain the proportion of class labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
n_features = X_train.shape[1] # Use the actual number of features
print(f"\nTraining features shape: {X_train.shape}")
print(f"Validation features shape: {X_val.shape}")


Training features shape: (1096, 590)
Validation features shape: (471, 590)


In [30]:
# --- Fitness Function (using Random Forest) ---
def fitness_function(individual, model, penalty):
    selected_indices = np.where(individual == 1)[0] # Get indices where bit is 1
    num_selected = len(selected_indices)

    #TODO: This validation can be done before calling this function!!!!!!!!!
    if num_selected == 0:
        return -np.inf # Heavily penalize empty feature set

    # Select corresponding columns from train/validation sets
    X_train_sel = X_train[:, selected_indices]
    X_val_sel = X_val[:, selected_indices]

    # Check for NaNs *before* fitting (added robustness)
    """ if np.isnan(X_train_sel).any() or np.isnan(y_train).any() or np.isnan(X_val_sel).any():
         # This case should ideally not happen after imputation, but good safeguard
         print("Warning: NaN detected in data subset for fitness evaluation.")
         return -np.inf # Penalize subsets leading to NaNs """
    
    try:        
        model.fit(X_train_sel, y_train)
        y_pred = model.predict(X_val_sel)
        mse = mean_squared_error(y_val, y_pred)
        # Fitness = Negative MSE (maximization) - Penalty for feature count
        fitness = -mse - penalty * num_selected
    except ValueError as e:
        # Catch potential errors during fitting/prediction if data issues remain
        print(f"Error during model fitting/prediction: {e}")
        fitness = -np.inf # Penalize if model fails
    return fitness

In [35]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

def evaluate_population(population, classif_model, penalty, n_jobs=-1, batch_size=10):
    print(f'\nEvaluating population fitness in parallel (batch_size={batch_size})..')

    # Wrapper to enable tqdm progress bar
    results = Parallel(n_jobs=n_jobs, batch_size=batch_size)(
        delayed(fitness_function)(ind, classif_model, penalty) 
        for ind in tqdm(population, desc="Fitness Evaluation")
    )

    return np.array(results)


In [31]:
#The expected number of selected features is d ⊆ n_features, this constraint must be set up since the first population
# to make more likely for the algorithm to focus on small subsets instead of larger ones in further generations.
#We set the chance of each gene being 1 in the initial population P to a value of d/D
def initialize_population(d, pop_size, n_features):
    probability_for_1 = d/n_features
    probability_for_0 = 1 - probability_for_1
    
    print(f'-Building Initial population of size: {pop_size} with approximately {d} features per individual')
    
    pop = np.random.choice([0, 1], size=(pop_size, n_features), p=[probability_for_0, probability_for_1])    
    
    # Ensure initial individuals are not all zeros
    for i in range(pop_size):
        if not pop[i].any:
            #then randomly sets one feature to 1.
            pop[i, np.random.randint(0, n_features)] = 1
    return pop

In [9]:
def tournament_selection(pop, fitnesses, k=3):
    # Handle potential -inf fitness values gracefully
    valid_indices = np.where(np.isfinite(fitnesses))[0]
    if len(valid_indices) == 0:
        # If no individuals have valid fitness, return random individuals to avoid crash
        print("Warning: No individuals with valid fitness in selection.")
        return pop[np.random.choice(len(pop), size=len(pop))]
    if len(valid_indices) < k:
        # If fewer valid individuals than tournament size, select randomly from valid ones
        selected_idx = np.random.choice(valid_indices, size=len(pop), replace=True) # Allow replacement
        return pop[selected_idx]

    selected = []
    for _ in range(len(pop)):
       # Ensure tournament participants are chosen only from valid individuals
       tournament_contenders_indices = np.random.choice(valid_indices, k, replace=False)
       winner_idx_in_contenders = np.argmax(fitnesses[tournament_contenders_indices])
       winner_original_idx = tournament_contenders_indices[winner_idx_in_contenders]
       selected.append(pop[winner_original_idx])
    return np.array(selected)

In [10]:
def uniform_crossover(parent1, parent2):
    mask = np.random.randint(0, 2, size=len(parent1), dtype=bool)
    child = np.where(mask, parent1, parent2)
    return child

def mutate(individual, mutation_rate):
    mutant = individual.copy()
    mutation_mask = np.random.rand(len(mutant)) < mutation_rate
    mutant[mutation_mask] = 1 - mutant[mutation_mask] # Flip bits
    # Ensure mutation doesn't result in an all-zero individual
    if np.sum(mutant) == 0 and len(mutant) > 0:
         mutant[np.random.randint(0, len(mutant))] = 1
    return mutant

In [32]:
# --- Local Search ---
# (This function remains unchanged structurally)
def local_search(individual, fitness_func, steps, model, penalty): # Uses tuned steps
    best = individual.copy()
    current_fitness = fitness_func(best, model, penalty)
    # Cannot perform local search on invalid individual
    if not np.isfinite(current_fitness):
        return best

    indices_to_try = np.arange(len(best))
    for _ in range(steps):
        np.random.shuffle(indices_to_try)
        improved_in_step = False
        for idx in indices_to_try:
            candidate = best.copy()
            candidate[idx] = 1 - candidate[idx] # Flip bit
            # Ensure candidate is not all zeros
            if np.sum(candidate) == 0:
                continue # Skip invalid (all zero) candidate
            candidate_fitness = fitness_func(candidate, model, penalty)
            # Only accept improvement if candidate_fitness is valid and better
            if np.isfinite(candidate_fitness) and candidate_fitness > current_fitness:
                best = candidate
                current_fitness = candidate_fitness
                improved_in_step = True
                break # Move to next step once an improvement is found (first improvement)
        if not improved_in_step:
            break # Stop if no single flip improved fitness in a full pass
    return best

In [36]:
# --- 5. Hybrid Genetic Algorithm for Feature Selection ---
# --- Main HGA Loop ---
# (This function remains unchanged structurally, but uses the tuned parameters)
#d_features: Desired features
def hybrid_ga_feature_selection(d_features, pop_size, n_gen, classif_model, penalty, elitism, n_elite, crossover_rate, mutation_rate,
                                 ls_top_k, local_search_steps):
    population = initialize_population(d_features, pop_size, n_features)
    best_fitnesses = []
    best_individual_overall = None
    best_fit_overall = -np.inf
    features_selected_history = []

    print("\nStarting Hybrid Genetic Algorithm...")
    for gen in range(n_gen): # Uses tuned N_GEN
        print(f'\n-Running Generation: {gen}----')
        
        # Evaluate fitness for the current population
        print(f'\nEvaluating population fitness..')
        #fitnesses = np.array([fitness_function(ind, classif_model, penalty) for ind in population])          
        fitnesses = evaluate_population(population, classif_model, penalty, n_jobs=-1, batch_size=10)      

        # Find best valid individual in generation
        print(f'\nFinding best valid individual..')

        valid_fitness_indices = np.where(np.isfinite(fitnesses))[0]
        if len(valid_fitness_indices) == 0:
            print(f"Warning: No valid individuals in generation {gen+1}. Stopping early.")
            # Keep the last known best if available, otherwise end with None
            best_fitnesses.append(best_fit_overall if np.isfinite(best_fit_overall) else np.nan)
            features_selected_history.append(np.sum(best_individual_overall) if best_individual_overall is not None else 0)
            break # Stop the GA run

        gen_best_idx_among_valid = np.argmax(fitnesses[valid_fitness_indices])
        gen_best_original_idx = valid_fitness_indices[gen_best_idx_among_valid]
        gen_best_fitness = fitnesses[gen_best_original_idx]

        # Update overall best
        if gen_best_fitness > best_fit_overall:
            best_fit_overall = gen_best_fitness
            best_individual_overall = population[gen_best_original_idx].copy()

        # Store history based on overall best
        best_fitnesses.append(best_fit_overall)
        features_selected_history.append(np.sum(best_individual_overall) if best_individual_overall is not None else 0)

        #--- GA Operators (Selection, Crossover, Mutation) ---
        # (These functions remain unchanged structurally)
        # Elitism
        if elitism:
             print(f'Aplying Elitism..')
             sorted_valid_indices = valid_fitness_indices[np.argsort(fitnesses[valid_fitness_indices])]
             elite_indices = sorted_valid_indices[-n_elite:]
             elites = population[elite_indices].copy()
        else:
             elites = np.empty((0, n_features), dtype=int)

        # Selection
        print(f'Selecting parents by {tournament_selection.__name__}...')
        selected_parents = tournament_selection(population, fitnesses)

        
        # Crossover & Mutation
        print(f'Applying Crossover and Mutation operators...')
        offspring = np.empty_like(population)
        for i in range(0, pop_size, 2): # Uses tuned POP_SIZE
            p1_idx, p2_idx = i, (i + 1) % pop_size
            parent1, parent2 = selected_parents[p1_idx], selected_parents[p2_idx]
            if np.random.rand() < crossover_rate:
                child1 = uniform_crossover(parent1, parent2)
                child2 = uniform_crossover(parent2, parent1)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            offspring[i] = mutate(child1, mutation_rate)
            if i + 1 < pop_size:
                offspring[i+1] = mutate(child2, mutation_rate)

        # Local Search on top K individuals (uses tuned LS_TOP_K)
        #fitnesses_offspring_pre_ls = np.array([fitness_function(ind, classif_model, penalty) for ind in offspring])
        fitnesses_offspring_pre_ls = evaluate_population(offspring, classif_model, penalty, n_jobs=-1, batch_size=10)      
        
        valid_offspring_indices = np.where(np.isfinite(fitnesses_offspring_pre_ls))[0]

        if len(valid_offspring_indices) > 0:
             sorted_valid_offspring_indices = valid_offspring_indices[np.argsort(fitnesses_offspring_pre_ls[valid_offspring_indices])]
             top_indices_for_ls = sorted_valid_offspring_indices[-ls_top_k:] # Uses tuned LS_TOP_K
             print(f'Applying Local Search...')
             for idx in top_indices_for_ls:
                 # Uses tuned LOCAL_SEARCH_STEPS inside the function call
                 offspring[idx] = local_search(offspring[idx], fitness_function, local_search_steps, classif_model, penalty) 

        # Replacement with Elitism        
        if elitism and len(elites) > 0:
            print(f'Obtaining offspring with Elitism...')
            #fitnesses_offspring_post_ls = np.array([fitness_function(ind, classif_model, penalty) for ind in offspring])
            fitnesses_offspring_post_ls = evaluate_population(offspring, classif_model, penalty, n_jobs=-1, batch_size=10)      

            valid_offspring_indices_post_ls = np.where(np.isfinite(fitnesses_offspring_post_ls))[0]
            if len(valid_offspring_indices_post_ls) >= n_elite:
                sorted_valid_offspring_indices_post_ls = valid_offspring_indices_post_ls[np.argsort(fitnesses_offspring_post_ls[valid_offspring_indices_post_ls])]
                worst_indices = sorted_valid_offspring_indices_post_ls[:n_elite]
                num_to_replace = min(len(worst_indices), len(elites))
                offspring[worst_indices[:num_to_replace]] = elites[:num_to_replace]

        population = offspring

        current_best_features = np.sum(best_individual_overall) if best_individual_overall is not None else 0
        print(f"Generation {gen+1}/{n_gen}: Best Fitness = {best_fit_overall:.4f}, Features Selected = {current_best_features}")

    print("Hybrid Genetic Algorithm finished.")
    return best_individual_overall, best_fitnesses, features_selected_history



In [37]:
# GA Parameters (TUNED for potentially faster runtime)
# --- TUNING POINT ---
POP_SIZE = 100 # Reduced from 30
N_GEN = 20    # Reduced from 30
# --- TUNING POINT ---
CROSSOVER_RATE = 0.8 # Kept the same
MUTATION_RATE = 0.1  # Kept the same
ELITISM = True       # Kept the same
N_ELITE = 1          # Kept the same
# --- TUNING POINT ---
LOCAL_SEARCH_STEPS = 2 # Reduced from 5
LS_TOP_K = 2          # Reduced from 3
# --- TUNING POINT ---
PENALTY_COEFF = 0.01   # Kept the same

DESIRED_FEATURES_NUMBER = 50

# DEfine the Classifier model: Random Forest
# --- TUNING POINT ---
# Use fewer estimators for significantly faster fitness evaluation
model = RandomForestRegressor(n_estimators=15, # Reduced from 50
                                random_state=42,
                                n_jobs=-1) # Use n_jobs=-1 for parallelization
# --- TUNING POINT ---

# --- 6. Run the Hybrid GA ---
best_solution, fitness_history, features_selected_history = hybrid_ga_feature_selection(DESIRED_FEATURES_NUMBER, POP_SIZE, N_GEN, model, PENALTY_COEFF, ELITISM, N_ELITE, CROSSOVER_RATE, MUTATION_RATE,
                                 LS_TOP_K, LOCAL_SEARCH_STEPS)

# --- 7. Results ---
if best_solution is not None:
    selected_feature_indices = np.where(best_solution == 1)[0]
    print("\n--- HGA Results ---")
    print("Best feature subset found:")
    print("Selected feature indices:", selected_feature_indices)
    # Use ACTUAL feature names from the header
    selected_names = [feature_names[i] for i in selected_feature_indices]
    print("Selected feature names:", selected_names)
    print("Number of features selected:", len(selected_feature_indices))
    print(f"Final Best Fitness: {fitness_history[-1]:.4f}")
else:
    print("\nNo solution found or algorithm stopped early.")

-Building Initial population of size: 100 with approximately 50 features per individual

Starting Hybrid Genetic Algorithm...

-Running Generation: 0----

Evaluating population fitness..

Evaluating population fitness in parallel (batch_size=10)..


Fitness Evaluation: 100%|██████████| 100/100 [00:04<00:00, 24.68it/s]



Finding best valid individual..
Aplying Elitism..
Selecting parents by tournament_selection...
Applying Crossover and Mutation operators...

Evaluating population fitness in parallel (batch_size=10)..


Fitness Evaluation: 100%|██████████| 100/100 [00:03<00:00, 32.75it/s]


Applying Local Search...
Obtaining offspring with Elitism...

Evaluating population fitness in parallel (batch_size=10)..


...

Generation 5/20: Best Fitness = -0.5854, Features Selected = 31

-Running Generation: 5----

Evaluating population fitness..

Evaluating population fitness in parallel (batch_size=10)..


Fitness Evaluation: 100%|██████████| 100/100 [00:16<00:00,  6.13it/s]



Finding best valid individual..
Aplying Elitism..
Selecting parents by tournament_selection...
Applying Crossover and Mutation operators...

Evaluating population fitness in parallel (batch_size=10)..


Fitness Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:

# --- 8. Plot Fitness Curve and Number of Features Selected ---
if best_solution is not None and fitness_history and features_selected_history:
    generations = range(1, len(fitness_history) + 1)
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(generations, fitness_history, marker='o', linestyle='-')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.title('HGA Convergence: Best Fitness per Generation')
    plt.grid(True)
    plt.subplot(1, 2, 2)
    plt.plot(generations, features_selected_history, marker='s', linestyle='-', color='orange')
    plt.xlabel('Generation')
    plt.ylabel('Number of Features Selected')
    plt.title('HGA: Number of Features in Best Solution per Generation')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Cannot plot results as no valid solution was found or history is empty.")
